In [1]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'

In [2]:
filename = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_numtext.csv'

In [44]:
m = csvigo.load({path = filename, mode = "large"})
print(#m, 'rows read in')

<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_numtext.csv	


<csv>	parsing done	
202216	rows read in	


In [43]:
function split(pString)
   local pPattern = " "
   local Table = {}  -- NOTE: use {n = 0} in Lua-5.0
   local fpat = "(.-)" .. pPattern
   local last_end = 1
   local s, e, cap = pString:find(fpat, 1)
   while s do
      if s ~= 1 or cap ~= "" then
     table.insert(Table, cap)
      end
      last_end = e + 1
      s, e, cap = pString:find(fpat, last_end)
   end
   if last_end <= #pString then
      cap = pString:sub(last_end)
      table.insert(Table, cap)
   end
   return Table
end

function grabn(x, n)
    local tmp = {}
    for k, v in pairs(x) do
        if k <= n then
            tmp[k] = tonumber(v)
        end
        if k > n then
            return tmp
        end
    end
end

In [10]:
N = 35
K = 5
embed_dim = 6

In [ ]:
out = {}
for k,v in pairs(m) do
    if k > 1 then
        out[k-1] = grabn(split(m[k][1]), 5)
    end
    if (k % N)==0 then
        print(k,'elements read out of ', #m)
        break
    end
end

--- getting the length of the dictionary
vocab_size = 0
for k,v in pairs(out) do
    vocab_size = math.max(vocab_size, math.max(table.unpack(v)))
    if (k % N)==0 then
        print(k,'elements read out of ', #m)
        break
    end
end

In [1]:
labels = torch.Tensor(torch.round(torch.rand(#out))):reshape(#out,1)
#labels, labels:sum(), labels:mean()

--- This is the correct format to input it
input = torch.LongTensor(out)
print(#input)

[string "labels = torch.Tensor(torch.round(torch.rand(..."]:2: unexpected symbol near '#': 

In [17]:
LT = nn.LookupTable(vocab_size, embed_dim)

-- For batch inputs, it's a little easier to start with sequence-length x batch-size tensor, so we transpose songData
myDataT = input:t()
batchLSTM = nn.Sequential()
batchLSTM:add(LT) -- will return a sequence-length x batch-size x embedDim tensor
batchLSTM:add(nn.SplitTable(1, 3)) -- splits into a sequence-length table with batch-size x embedDim entries
print(batchLSTM:forward(myDataT)) -- sanity check
-- now let's add the LSTM stuff
batchLSTM:add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
batchLSTM:add(nn.SelectTable(-1)) -- selects last state of the LSTM
batchLSTM:add(nn.Linear(embed_dim, 1)) -- map last state to a score for classification
batchLSTM:add(nn.Sigmoid()) -- convert score to a probability
myPreds = batchLSTM:forward(myDataT)
print(myPreds)

-- we can now call :backward() as follows
bceCrit = nn.BCECriterion()
loss = bceCrit:forward(myPreds, labels)
dLdPreds = bceCrit:backward(myPreds, labels)
batchLSTM:backward(myDataT, dLdPreds)

print(loss)

...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 1 module of nn.Sequential:
...vierarceo/torch/install/share/lua/5.1/nn/LookupTable.lua:75: index out of range at /Users/franciscojavierarceo/torch/pkg/torch/lib/TH/generic/THTensorMath.c:156
stack traceback:
	[C]: in function 'index'
	...vierarceo/torch/install/share/lua/5.1/nn/LookupTable.lua:75: in function <...vierarceo/torch/install/share/lua/5.1/nn/LookupTable.lua:68>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function 'forward'
	[string "embed_dim = 6..."]:9: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/i

In [54]:
zc = 0
for k, v in pairs(out) do
    for j,l in pairs(v) do
        if l == 0 then
            zc = zc + 1
            print(k, j)
        end
    end
end

34	5	


In [38]:
for j,l in pairs(out[34]) do
    print(j,l)
end

1	425	
2	336	
3	335	
4	258	
5	0	


In [42]:
for j,l in pairs(out[34]) do
    if l == '0' then
        print(j,l)
    end
end

5	0	
